# Recommender Evaluation Framework

Aqui, vamos definir as métricas que vamos utilizar para medir a qualidade das recomendações que vamos fazer.

Para isso, vamos realizar uma recomendação teste:

In [1]:
import numpy as np
import pandas as pd
import feather as ft
import os
import mpd

Selecionando uma playlist exemplo para testar as métricas:

In [2]:
PLAYLISTS_FILE = "data/train-test/playlists_train.fthr"
RANDOM_STATE = 433

In [3]:
df_playlists = ft.read_dataframe(PLAYLISTS_FILE)
example_playlist = df_playlists.sample(1, random_state=RANDOM_STATE)
del df_playlists

In [4]:
example_playlist

,pid,name,collaborative,modified_at,duration_ms,num_tracks,num_albums,num_artists,num_followers,num_edits
8009,766126,Party,0,1504310400,62857995,234,199,135,1,25


In [5]:
example_pid = example_playlist["pid"].values[0]
mpd.show_playlist(example_pid)

=== 766126 ===
Party
  followers      1    
  modified       2017-09-01
  edits          25   
  collaborative  false
  num_artists    135  
  num_tracks     234  
  duration min   1047.63

  # | track                          | album                                    | artist              
----------------------------------------------------------------------------------------------------
  0 | Radar Love                     | Moontan                                  | Golden Earring      
  1 | Sex on Fire                    | Only By The Night                        | Kings of Leon       
  2 | Black Hole Sun                 | Superunknown                             | Soundgarden         
  3 | The Middle                     | Bleed American                           | Jimmy Eat World     
  4 | It's Been Awhile - Explicit L  | Break The Cycle                          | Staind              
  5 | Keep On Swinging               | Head Down                                | Rival Son

In [6]:
df_pt = ft.read_dataframe('data/train-test/playlist_track_train.fthr')
df_example_playlist = df_pt[df_pt["pid"]==example_pid].copy()

In [7]:
TEST_FRAC = 0.2

In [8]:
test_idx = int(df_example_playlist.shape[0]*(1-TEST_FRAC))
df_example_playlist_base = df_example_playlist.iloc[:test_idx]
df_example_playlist_ho   = df_example_playlist.iloc[test_idx:]

Agora temos uma playlist de exemplo separada em base (80%) e hold-out (20%), que poderemos usar para testar algumas recomendações. Para isso, vamos gerar uma previsão com algumas faixas relevates e outras aleatórias retiradas das outras playlists a que temos acesso.

In [9]:
# 20 relevant tracks
relevant_tracks = df_example_playlist_ho["tid"].sample(20).values

In [10]:
# 480 random selections
random_tracks = df_pt["tid"].drop_duplicates().sample(480).values

In [11]:
recomendations = np.concatenate((relevant_tracks, random_tracks))
np.random.shuffle(recomendations)

## 1 - R-precision

O número de faixas relevantes obtidas dividido pelo numero de faixas relevantes conhecidas (número das faixas para teste)

In [12]:
def r_precision(ho_tracks, recommendations):
    
    relevant_predicitons = len(set(ho_tracks).intersection(set(recommendations)))
    
    return relevant_predicitons / len(ho_tracks)

In [13]:
r_precision(df_example_playlist_ho["tid"], recomendations)

0.425531914893617

## 2 - NDCG
*Normalized Discounted Cumulative Gain*

Para medir a qualidade do ranking das recomendações realizado pelo sistema.

In [14]:
def dcg(r):

    r = np.asfarray(r)
    
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))  
    
    return 0.


def ndcg(r):
   
    #ideal dcg
    idcg = dcg(sorted(r, reverse=True))
    
    if not idcg:
        return 0.
    
    return dcg(r) / idcg

def ncdg_playlist(ho_tracks, recommendations):
    
    return ndcg(np.in1d(recomendations, ho_tracks))

In [15]:
ncdg_playlist(df_example_playlist_ho["tid"], recomendations)

0.42273973213326255

## 3 - Recommended Songs Clicks

Métrica que também leva em consideração a ordem das recomendações, mas entra no detalhe da implementação do Spotify, que apresenta 10 faixas, que podem ser atualizadas pelo usuário. Essa métrica indica o número de páginas necessárias até que se encontre uma faixa relevante.

In [16]:
def rsc(r):
    
    for i in range(0, len(r), 10):
        if sum(r[i:i + 10]):
            return (i//10+1)
    return 51

def rsc_playlist(ho_tracks, recommendations):
    
    return rsc(np.in1d(recomendations, df_example_playlist_ho["tid"]))

In [17]:
rsc_playlist(df_example_playlist_ho["tid"], recomendations)

1

In [18]:
# exemplo em que a primeira sugestão relevante aparece na 4 página
recs = [0]*36 + [1] + [0]*463

In [19]:
rsc(recs)

4

<hr>